In [ ]:
!pip install catboost
!pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 4.8 MB/s eta 0:00:00
ERROR: Operation cancelled by user
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 5.2 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [ ]:
pip install --upgrade scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 20.8 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import mutual_info_classif
import seaborn as sns
import matplotlib.pyplot as plt
from termcolor import colored
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
#from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, recall_score, precision_score, confusion_matrix, classification_report, RocCurveDisplay

In [ ]:
cardio=pd.read_csv("/content/cardio_train.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/content/cardio_train.csv'

In [ ]:
cardio

In [ ]:
split_data = []

# Iterate over each line in the data
for line in cardio:
    # Split the line by semicolon and strip any whitespace
    split_line = [elem.strip() for elem in line.split(';')]
    # Add the split line to our data list
    split_data.append(split_line)

# Convert the list of data into a DataFrame
# We are skipping the first row because it contains the column names
data_df_manual1 = pd.DataFrame(split_data[1:], columns=split_data[0])

# Display the first few entries to verify the DataFrame structure
data_df_manual1.head()


In [ ]:
split_data = []
expected_num_columns = None

# Iterate over each line in the data
for index, line in enumerate(cardio['id;age;gender;height;weight;ap_hi;ap_lo;cholesterol;gluc;smoke;alco;active;cardio']):
    # Split the line by semicolon and strip any whitespace
    split_line = [elem.strip() for elem in line.split(';')]

    # Set expected number of columns based on the first line
    if index == 0:
        expected_num_columns = len(split_line)

    # Check if the current line has the expected number of columns
    if len(split_line) == expected_num_columns:
        split_data.append(split_line)
    else:
        print(f"Data format error in line {index}: {line}")

In [ ]:
cardio = pd.DataFrame(split_data, columns=data_df_manual1.columns)

In [ ]:
cardio.drop(['id'],inplace=True,axis=1)

In [ ]:
cardio['age'] = np.floor(cardio['age'].astype(float) / 365.25).astype(int)

In [ ]:
cardio.info()

In [ ]:
cardio.describe(include='object')

In [ ]:
cardio['ap_lo'] = cardio['ap_lo'].astype(int)

In [ ]:
cardio['ap_hi'] = cardio['ap_hi'].astype(int)

In [ ]:
cardio['height'] = cardio['height'].astype(int)

In [ ]:
cardio['weight'] = cardio['weight'].astype(float)

In [ ]:
cardio.describe()

In [ ]:
def check_nulls(df):
  null_counts = {}
  for column in df.columns:
        count = df[column].isnull().sum()
        null_counts[column] = count
        print(f" {column} column : has {count} null value.")

  return null_counts

In [ ]:
k = 5
X = data_scaled_df.drop(columns=['cardio'])
y = data_scaled_df['cardio']
# Initialize SelectKBest with mutual information score
selector = SelectKBest(score_func=mutual_info_classif, k=k)

# Fit selector to training data
selector.fit(X, y)

# Get selected feature indices
selected_indices = selector.get_support(indices=True)

# Get names of selected features
selected_features = X.columns[selected_indices]

print("Selected features:", selected_features)

In [ ]:
x=cardio.drop(['cardio'],axis=1)
y=cardio['cardio']
mutual_info_scores = mutual_info_classif(x, y)

# Create a DataFrame to store feature names and their corresponding scores
feature_scores_df = pd.DataFrame({'Feature': x.columns, 'Mutual Information Score': mutual_info_scores})

# Sort the DataFrame by mutual information scores in descending order
feature_scores_df = feature_scores_df.sort_values(by='Mutual Information Score', ascending=False)

In [ ]:
feature_scores_df

In [ ]:
null_info = check_nulls(cardio)

In [ ]:
#outliers
cardio[cardio["ap_hi"]<0]
cardio["ap_hi"]=cardio["ap_hi"].abs()
cardio[cardio["ap_lo"]<0]
cardio["ap_lo"]=cardio["ap_lo"].abs()

In [ ]:
def detect_outliers(df):
    outlier_indices = {}
    for column in df.columns:
        if df[column].dtype != 'object':  # Exclude non-numeric columns
            # Calculate Q1 (25th percentile) and Q3 (75th percentile) of the column
            Q1 = df[column].quantile(0.25)
            Q3 = df[column].quantile(0.75)
            # Define the Interquartile Range (IQR)
            IQR = Q3 - Q1
            # Define outlier bounds
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            # Find the outliers
            outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
            # Store the indices of the outliers if any
            if not outliers.empty:
                outlier_indices[column] = outliers.index.tolist()

    return outlier_indices

In [ ]:
outliers_dict = detect_outliers(cardio)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def detect_and_plot_outliers(df, columns):
    """
    Detect outliers in specified columns of a DataFrame based on the IQR method and plot the results.

    Parameters:
    df (pandas.DataFrame): The DataFrame to check for outliers.
    columns (list): List of column names to check for outliers.

    Returns:
    dict: A dictionary with column names as keys and lists of outlier indices as values.
    """
    outlier_indices = {}

    # Set up the matplotlib figure
    num_plots = len(columns)
    fig, axes = plt.subplots(num_plots, figsize=(10, num_plots * 5))

    if num_plots == 1:
        axes = [axes]  # Make it iterable

    for ax, column in zip(axes, columns):
        if column in df.columns and df[column].dtype != 'object':  # Check for non-object column
            # Calculate Q1 (25th percentile) and Q3 (75th percentile) of the column
            Q1 = df[column].quantile(0.25)
            Q3 = df[column].quantile(0.75)
            # Define the Interquartile Range (IQR)
            IQR = Q3 - Q1
            # Define outlier bounds
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            # Find the outliers
            outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
            # Store the indices of the outliers if any
            if not outliers.empty:
                outlier_indices[column] = outliers.index.tolist()

            # Plot
            sns.boxplot(x=df[column], ax=ax)

            # Determine whisker's end locations for better visualization
            whisker_props = dict(linestyle='-', linewidth=2, color='blue')
            flier_props = dict(marker='o', markersize=5, linestyle='none', markeredgecolor='red')

            sns.boxplot(x=df[column], ax=ax, whis=1.5, showfliers=True,
                        whiskerprops=whisker_props, flierprops=flier_props)

            ax.set_title(f'Outlier Detection in "{column}"')
            ax.set_xlabel(column)

            # Set x-axis to logarithmic scale if the data has a large range
            # ax.set_xscale('log')
            ax.legend()

    plt.tight_layout()
    plt.show()

    return outlier_indices

# Example usage:
# Assuming 'data_df' is your DataFrame
# data_df = pd.DataFrame({
#     'A': [1, 2, 3, 4, 100],  # 100 is an outlier
#     'B': [10, 20, 30, 40, 50],  # No outliers
#     'C': [100, 200, 300, -


In [ ]:
outliers_dict = detect_and_plot_outliers(cardio, ['ap_hi','ap_lo','weight','height'])

##REMOVING OUTLIERS


In [ ]:
'''def standard_deviation_removal(data, num_std_dev=4):
    if data.dtype.kind in 'bifc':  # Checks if the column is numeric
        mean = data.mean()
        std_dev = data.std()
        return data[(data > mean - num_std_dev * std_dev) & (data < mean + num_std_dev * std_dev)]
    else:
        return data'''

In [ ]:
'''def quantile_based_removal(data, lower_quantile=0.01, upper_quantile=0.99):
    if data.dtype.kind in 'bifc':  # Checks if the column is numeric
        qlow = data.quantile(lower_quantile)
        qhigh = data.quantile(upper_quantile)
        return data[(data > qlow) & (data < qhigh)]
    else:
        return data'''

In [ ]:
'''def apply_outlier_methods(df):
    # Apply each method to each column and store the result
    for method in [quantile_based_removal]:
        for column in df.columns:
            original_data = df[column]
            cleaned_data = method(df[column])
            # Report which rows were removed
            removed_rows = original_data[~original_data.index.isin(cleaned_data.index)]
            if not removed_rows.empty:
            # Update the DataFrame column
            df[column] = cleaned_data
    return df'''

In [ ]:
cardio = apply_outlier_methods(cardio)

In [ ]:
cardio

In [ ]:
cardio.info()

In [ ]:
def fill_nan_with_mean(df):
    """
    Fills NaN values in numeric columns of a DataFrame with the mean of each column.

    Parameters:
    df (pandas.DataFrame): The DataFrame to process.

    Returns:
    pandas.DataFrame: The DataFrame with NaN values replaced by mean in numeric columns.
    """
    # Iterate over each column in the DataFrame
    for column in df.columns:
        # Check if the column is numeric
        if df[column].dtype in ['int64', 'float64']:
            # Calculate the mean of the column, excluding NaN values
            column_mean = df[column].mean()
            # Replace NaN values with the mean
            df[column].fillna(column_mean, inplace=True)
    return df

In [ ]:
cardio = fill_nan_with_mean(cardio)

In [ ]:
cardio.info()

In [ ]:
cardio=cardio[cardio['ap_hi'] <= 250]

In [ ]:
cardio=cardio[cardio['ap_lo'] <= 250]

In [ ]:
### removing height outlier
cardio=cardio[cardio['height'] >= 100]

In [ ]:
cardio=cardio[cardio['weight'] >= 35]

In [ ]:
cardio=cardio[cardio['weight'] <= 175]

In [ ]:
cardio=cardio[cardio['height'] <= 205]

In [ ]:
cardio=cardio[cardio['ap_hi'] >= 25]

In [ ]:
cardio=cardio[cardio['ap_lo'] >= 15]

In [ ]:
outliers_dict = detect_and_plot_outliers(cardio, ['ap_hi','ap_lo','height','weight'])

In [ ]:
cardio=cleaned_data

In [ ]:
cardio.describe()

In [ ]:
cor = cardio.corr()
plt.figure(figsize=(12, 8))
cmap = sns.cubehelix_palette(start=2, rot=0, dark=0, light=.95, reverse=True, as_cmap=True)
sns.heatmap(cor, cmap="Blues", annot=True)

plt.title('Correlation Heatmap', fontsize=16)
plt.show()

In [ ]:
correlation_matrix = cardio.corr()

#We only take the correlations with 'target'.
target_correlation = correlation_matrix['cardio'].sort_values(ascending=False)
target_correlation = target_correlation.drop("cardio")

plt.figure(figsize=(8, 4))
sns.heatmap(target_correlation.to_frame(), cmap="coolwarm", annot=True, vmin=-1, vmax=1)
plt.title("Correlations with Target Variable")
plt.show()

In [ ]:
def categorize_age(df, age_column, bins, labels):
  if age_column not in df.columns:
        raise ValueError(f"The column '{age_column}' does not exist in the DataFrame.")

    # Use pd.cut to categorize ages
  df['age_category'] = pd.cut(df[age_column], bins=bins, labels=labels,right=False, include_lowest=True)

  return df

In [ ]:
bins = [18, 30, 40, 50, 60, 101]
labels = ['18-29', '30-39', '40-49', '50-59', '60-100']
cardio = categorize_age(cardio, 'age', bins, labels)

In [ ]:
cardio.info()

In [ ]:
def add_bmi_category(df, height_column, weight_column):
  df['BMI'] = df[weight_column] / (df[height_column] / 100) ** 2

    # Define BMI categories according to common thresholds
  conditions = [
        (df['BMI'] < 18.5),
        (df['BMI'] >= 18.5) & (df['BMI'] < 25),
        (df['BMI'] >= 25) & (df['BMI'] < 30),
        (df['BMI'] >= 30)
    ]
  categories = ['Underweight', 'Normal', 'Overweight', 'Obese']

    # Use numpy.select to apply the conditions to categories
  df['BMI_Category'] = np.select(conditions, categories, default='Unknown')

  return df

In [ ]:
cardio = add_bmi_category(cardio, 'height', 'weight')

In [ ]:
cardio

In [ ]:
def categorize_blood_pressure(df, sys_column, dia_column):
  conditions = [
        (df[sys_column] < 120) & (df[dia_column] < 80),
        (df[sys_column] >= 120) & (df[sys_column] < 130) & (df[dia_column] < 80),
        ((df[sys_column] >= 130) & (df[sys_column] < 140)) | ((df[dia_column] >= 80) & (df[dia_column] < 89)),
        (df[sys_column] >= 140) | (df[dia_column] >= 90),
        (df[sys_column] > 180) | (df[dia_column] > 120)
    ]

  categories = [
        'Normal',
        'Elevated',
        'High Blood Pressure (Hypertension) Stage 1',
        'High Blood Pressure (Hypertension) Stage 2',
        'Hypertensive Crisis'
    ]

  df['BP_Category'] = np.select(conditions, categories, default='Unknown')

  return df

In [ ]:
cardio = categorize_blood_pressure(cardio, 'ap_hi', 'ap_lo')

In [ ]:
cardio['BP_Category'].value_counts()

In [ ]:
cardio

In [ ]:
def grab_col_names(dataframe, cat_th=10, car_th=20):

    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]


    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')

    return cat_cols, num_cols, cat_but_car

In [ ]:
cat_cols, num_cols, cat_but_car = grab_col_names(cardio)
print(f"\n{colored('Numerical Columns:','blue', attrs=['reverse'])} {num_cols}\n\n\n{colored('Categorical Columns:','magenta', attrs=['reverse'])} {cat_cols}\n\n\n"
        f"{colored('Cardinal Columns:','cyan', attrs=['reverse'])}{cat_but_car}\n")

In [ ]:
def cat_summary(dataframe, col_name, plot=False):
    display(pd.DataFrame({col_name: dataframe[col_name].value_counts(),
                        "Ratio": 100 * dataframe[col_name].value_counts() / len(dataframe)}))

    if plot:
        sns.countplot(x=dataframe[col_name], data=dataframe)
        plt.show()

In [ ]:
for col in cat_cols:
    cat_summary(cardio, col)

In [ ]:
def num_summary(dataframe, numerical_col, plot=False):
    if plot:
        dataframe[numerical_col].hist(bins=200)
        plt.xlabel(numerical_col)
        plt.title(numerical_col)
        plt.show(block=True)

In [ ]:
for col in num_cols:
    num_summary(cardio, col, True)

In [ ]:
cardio

In [ ]:
cardio.info()

In [ ]:
def convert_categorical_to_onehot(df):
    """
    Converts all categorical columns in the dataframe to one-hot encoded columns.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing the data.

    Returns:
    pandas.DataFrame: A DataFrame with original non-categorical columns and added one-hot encoded columns.
    """
    # Identify categorical columns (assume categorical columns are of type 'object' or 'category')
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns

    # If there are no categorical columns, just return the original DataFrame
    if not categorical_cols.tolist():
        return df

    # Initialize the OneHotEncoder
    ohe = OneHotEncoder(handle_unknown='ignore')

    # Apply OneHotEncoder to categorical columns
    ohe_result = ohe.fit_transform(df[categorical_cols])

    # Convert the result to a DataFrame with appropriate column names
    ohe_df = pd.DataFrame(ohe_result.toarray(), columns=ohe.get_feature_names_out(categorical_cols), index=df.index)

    # Debugging: Check the shapes to see if they match
    print(f"Shape of OHE result array: {ohe_result.shape}")
    print(f"Number of OHE column names: {len(ohe.get_feature_names_out(categorical_cols))}")

    # Drop original categorical columns and concatenate the OHE DataFrame
    df_dropped = df.drop(columns=categorical_cols)
    df_encoded = pd.concat([df_dropped, ohe_df], axis=1)

    return df_encoded


In [ ]:
x=cardio.drop('cardio',axis=1)

In [ ]:
onehot = convert_categorical_to_onehot(x)

In [ ]:
onehot

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
cardio.shape

In [ ]:
onehot.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(onehot, cardio['cardio'], test_size=0.2, random_state=42)

In [ ]:
onehot

In [ ]:
scaler = StandardScaler()
# Fit the scaler to the data and transform it
#scaled_features = scaler.fit_transform(onehot)
# Convert the scaled data back to a DataFrame for readability
#scaled_onehot = pd.DataFrame(scaled_features, columns=onehot.columns)
scaler.fit(X_train)
x_train=scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
svm_model = SVC(kernel='linear')

In [ ]:
svm_model.fit(X_train, y_train)

In [ ]:
# Predict on the test data
y_pred = svm_model.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
clf_report = classification_report(y_test, y_pred)

In [ ]:
v=list(y_pred)

In [ ]:
len(v)

In [ ]:
X_test

In [ ]:
!pip install ydata_profiling

In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(scaled_onehot, title="Pandas Profiling Report")
profile

In [ ]:
scaled_onehot.drop_duplicates(inplace=True)

In [ ]:
data_df

In [ ]:
!pip install pycaret

In [ ]:
scaled_onehot['target'] = y

In [ ]:
scaled_onehot

In [ ]:
from pycaret.classification import *
s = setup(scaled_onehot,target='target', train_size = 0.8, fold = 3)

In [ ]:
best = compare_models()

In [ ]:
!pip install lightgbm

In [ ]:
X = data_df
y = cardio['cardio']

In [ ]:
y.info()

In [ ]:
lgbm_model = LGBMClassifier()

In [ ]:
lgbm_model_= cross_validate(lgbm_model, data_df, y, cv = 10, scoring = ["accuracy", "f1", "roc_auc"])

In [ ]:
lgbm_model_["test_accuracy"].mean()